In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier


In [2]:
#check datatypes
def check_dtypes2(dataframe):
    test =dataframe.dtypes
    x=0
    for i in range (len(dataframe.columns)):
        print("COL NAME: ",dataframe.columns[i])
        print("  TYPE: ",test[x])
        print("  % OF UNIQUE VALUES: ", (len(pd.unique(dataframe[dataframe.columns[i]]))/len(dataframe[dataframe.columns[i]]))*100 )
        print("  Values: ",pd.unique(dataframe[dataframe.columns[i]]))
        print("  SAMPLE: ",dataframe.iloc[0][dataframe.columns[i]])
        print()
        x+=1
    return()
#dealing with null values

#null values info
def null_find(dataframe):
    null_df = pd.DataFrame(round(dataframe.isna().sum()/len(dataframe),4)*100)
    null_df = null_df.reset_index()
    null_df.columns = ['header','percentage_nulls']
    return(null_df)
           
#null values reduction
def null_reduction(dataframe,porcentage):
    null_df=null_find(dataframe)
    columns_drop = null_df[null_df['percentage_nulls']>porcentage]['header']
    return dataframe.drop(columns_drop,axis=1)

# print non zero null columns

def print_nan(dataframe):
    null_df = null_find(dataframe)
    null_df = null_df[null_df['percentage_nulls']>0]
    print(null_df)
    return()


#scaler
def scaleColumns(df, cols_to_scale):
    scaler = preprocessing.StandardScaler()

    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df


Read that data into Python and call the dataframe churnData.

In [3]:
churnData = pd.read_csv('./files_for_lab/Customer-Churn.csv')
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

In [4]:
check_dtypes2(churnData)

COL NAME:  gender
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Female' 'Male']
  SAMPLE:  Female

COL NAME:  SeniorCitizen
  TYPE:  int64
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  [0 1]
  SAMPLE:  0

COL NAME:  Partner
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Yes' 'No']
  SAMPLE:  Yes

COL NAME:  Dependents
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  tenure
  TYPE:  int64
  % OF UNIQUE VALUES:  1.0364901320460032
  Values:  [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
  SAMPLE:  1

COL NAME:  PhoneService
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  OnlineSecurity
  TYPE:  object
  % OF UNIQUE VALUES:  0.0425954848

()

In [5]:
churnData['TotalCharges']= pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [6]:
check_dtypes2(churnData)

COL NAME:  gender
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Female' 'Male']
  SAMPLE:  Female

COL NAME:  SeniorCitizen
  TYPE:  int64
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  [0 1]
  SAMPLE:  0

COL NAME:  Partner
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['Yes' 'No']
  SAMPLE:  Yes

COL NAME:  Dependents
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  tenure
  TYPE:  int64
  % OF UNIQUE VALUES:  1.0364901320460032
  Values:  [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
  SAMPLE:  1

COL NAME:  PhoneService
  TYPE:  object
  % OF UNIQUE VALUES:  0.02839698991906858
  Values:  ['No' 'Yes']
  SAMPLE:  No

COL NAME:  OnlineSecurity
  TYPE:  object
  % OF UNIQUE VALUES:  0.0425954848

()

Check for null values in the dataframe. Replace the null values.

In [7]:
null_find(churnData)

,header,percentage_nulls
0,gender,0.00
1,SeniorCitizen,0.00
2,Partner,0.00
3,Dependents,0.00
4,tenure,0.00
5,PhoneService,0.00
6,OnlineSecurity,0.00
7,OnlineBackup,0.00
8,DeviceProtection,0.00
9,TechSupport,0.00


In [8]:
churnData.dropna(subset=['TotalCharges'], inplace = True)

In [9]:
churnData.reset_index(drop=True, inplace = True)

In [10]:
null_find(churnData)

,header,percentage_nulls
0,gender,0.0
1,SeniorCitizen,0.0
2,Partner,0.0
3,Dependents,0.0
4,tenure,0.0
5,PhoneService,0.0
6,OnlineSecurity,0.0
7,OnlineBackup,0.0
8,DeviceProtection,0.0
9,TechSupport,0.0


Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

    Scale the features either by using normalizer or a standard scaler.

In [11]:
churnData = scaleColumns(churnData, ['SeniorCitizen','MonthlyCharges','TotalCharges'])


In [12]:
churnData.SeniorCitizen

0      -0.440327
1      -0.440327
2      -0.440327
3      -0.440327
4      -0.440327
          ...   
7027   -0.440327
7028   -0.440327
7029   -0.440327
7030    2.271039
7031   -0.440327
Name: SeniorCitizen, Length: 7032, dtype: float64

In [13]:
churnData['Churn'].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.

# Lab | Cross Validation

In [14]:
Xf=churnData.drop(['Churn'], axis=1)
yf=churnData['Churn']
Xf_num=Xf.select_dtypes(include = np.number)
Xf_cat = pd.get_dummies(Xf.select_dtypes(include = 'object'), drop_first=True)
Xf=pd.concat([Xf_num, Xf_cat], axis=1)

# Apply SMOTE for upsampling the data

In [15]:
from imblearn.over_sampling import SMOTE

# SMOTE only can handle numerical features. Therefore, if we want to use categoricals they 
# will need to be dummyfied. However, for sake of simplicity we will not include the categoricals.

data = pd.concat([Xf,yf],axis=1)
smote = SMOTE()
y = data['Churn']
X = data.drop(['Churn'], axis=1)
X_sm, y_sm = smote.fit_resample(X, y)

unique, counts = np.unique(y_sm, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 5163]
 ['Yes' 5163]]


# Use logistic regression to fit the model and compute the accuracy of the model.

In [16]:
Xf=data.drop(['Churn'], axis=1)
yf=data['Churn']
    
X_train, X_test, y_train, y_test = train_test_split(Xf, yf, test_size=0.3, random_state=42)

In [17]:
model = LogisticRegression(max_iter = 500)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.83      0.89      0.86      1549
         Yes       0.63      0.51      0.57       561

    accuracy                           0.79      2110
   macro avg       0.73      0.70      0.71      2110
weighted avg       0.78      0.79      0.78      2110



# Use decision tree classifier to fit the model and compute the accuracy of the model.

In [18]:
model = DecisionTreeClassifier(random_state=0)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.82      0.79      0.81      1549
         Yes       0.47      0.51      0.49       561

    accuracy                           0.72      2110
   macro avg       0.65      0.65      0.65      2110
weighted avg       0.73      0.72      0.72      2110



# -Compare the accuracies of the two models.

Logistic regression is better in this case with higher scores in every metric

# Apply TomekLinks for downsampling

    It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.

In [19]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_resample(Xf, yf)


unique, counts = np.unique(y_tl, return_counts=True)
print(np.asarray((unique, counts)).T)

C:\Users\Kyre\AppData\Roaming\Python\Python39\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


[['No' 4684]
 ['Yes' 1869]]


# Use logistic regression to fit the model and compute the accuracy of the model.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_tl, y_tl, test_size=0.3, random_state=42)

In [21]:
model = LogisticRegression(max_iter = 500)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.84      0.88      0.86      1376
         Yes       0.68      0.60      0.64       590

    accuracy                           0.80      1966
   macro avg       0.76      0.74      0.75      1966
weighted avg       0.79      0.80      0.79      1966



# Use decision tree classifier to fit the model and compute the accuracy of the model.

In [22]:
model = DecisionTreeClassifier(random_state=0)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.82      0.82      0.82      1376
         Yes       0.57      0.57      0.57       590

    accuracy                           0.74      1966
   macro avg       0.70      0.70      0.70      1966
weighted avg       0.74      0.74      0.74      1966



# Compare the accuracies of the two models.
logistic regression is better again

# You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.

In [23]:
tl = TomekLinks('majority')
X_tl2, y_tl2 = tl.fit_resample(X_tl, y_tl)


unique, counts = np.unique(y_tl2, return_counts=True)
print(np.asarray((unique, counts)).T)

C:\Users\Kyre\AppData\Roaming\Python\Python39\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


[['No' 4554]
 ['Yes' 1869]]
